# 1. 环境配置

## 1.1 python 环境准备

In [5]:
! pip install openai==2.11.0 dashscope==1.25.4 langchain-classic==1.0.0 langchain==1.1.3 langchain-community==0.4.1 langchain-openai==1.1.3

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## 1.2 大模型密钥准备

请根据第一章内容获取相关平台的 API KEY，如若未在系统变量中填入，请将 API_KEY 信息写入以下代码（若已设置请忽略）：

In [6]:
import os

# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxx"
# os.environ["DASHSCOPE_API_KEY"] = "sk-yyyyyyyy"

## 1.3 大模型代码准备
在 5.3 章节中，我们已经准备好了大模型的代码，这里我们直接使用即可。

In [12]:
from langchain_community.chat_models import ChatTongyi
import os

llm = ChatTongyi(
  api_key=os.environ.get("DASHSCOPE_API_KEY"), 
  model="qwen-max")

from langgraph.checkpoint.memory import InMemorySaver 
memory = InMemorySaver()

system_prompt = "You are a helpful assistant"

from langchain.tools import tool

@tool
def calculate(what: str) -> str:
  """
  calculate:
  e.g. calculate: 4 * 7 / 3
  Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary
  """
  return str(eval(what))

@tool
def average_dog_weight(name: str) -> str:
  """
  average_dog_weight:
  e.g. average_dog_weight: Collie
  returns average weight of a dog when given the breed
  """
  name = name.lower()
  if "scottish terrier" in name:
    return "Scottish Terriers average 20 lbs"
  elif "border collie" in name:
    return "A Border Collie's average weight is 37 lbs"
  elif "toy poodle" in name:
    return "A Toy Poodle's average weight is 7 lbs"
  else:
    return "An average dog weighs 50 lbs"
  
tools = [calculate, average_dog_weight]


from langchain.agents import create_agent
agent = create_agent(model=llm, tools=tools, system_prompt=system_prompt, checkpointer=memory)

# 2. 输出模式

## 2.1 流式输出

假如我们希望能够逐步看到内部的变化，而不是一次性全部打印，可以使用流式输出实现。

对于智能体而言，流式输出最大的价值并非仅仅只是一个个 token 的进行输出，更重要的是能够实现一段段动态的展示信息，从而能够更方便的检测智能体内部的变化。

在 LangGraph 中，可以有不同的流式输出模式，如：
- "messages"：模型逐个 token 输出最终结果
- "values"：全局状态快照（展示整体的更新）
- "updates"：智能体执行进度（每一个节点都进行更新）
- "custom"：自定义事件（比如“正在读取数据…”）

传入时可以通过 `for chunk in agent.stream(..., stream_mode="updates"):` 进行运行模式的设置。

### 2.1.1 messages 模式

In [13]:
for token, metadata in agent.stream(
 {
  "messages": [{
   "role": "user",
   "content": "I have 2 dogs, a border collie and a scottish terrier. What is their combined weight? Could you double it?"
  }]
 },
 config={
  "configurable": {"thread_id": "user_1"},
 },
 stream_mode="messages" 
):
  print(f"{token.content}", end=" ")    # 结尾添加一个空格可以更明显看到流式输出的形式

 A Border Collie's average weight is 37 lbs Scottish Terriers average 20 lbs The average weight of a Border Collie is 37 lbs and the average weight of a Scottish Terrier is 20 lbs. Combined, their average weights would total 57 lbs. 

Now, let's double this combined weight. 114 The doubled combined average weight of your Border Collie and Scottish Terrier is 114 lbs. 

### 2.1.2 value 模式

 values 模式下，流式输出的调用方式为：

In [14]:
for step in agent.stream(
 {
  "messages": [{
   "role": "user",
   "content": "I have 2 dogs, a border collie and a scottish terrier. What is their combined weight? Could you double it?"
  }] # 提问的问题
 },
 config={
  "configurable": {"thread_id": "user_1"} # 短期记忆
 },
 stream_mode="values" # 可更换为 messages 或自定义方法
):
  step["messages"][-1].pretty_print()

================================ Human Message =================================

I have 2 dogs, a border collie and a scottish terrier. What is their combined weight? Could you double it?
================================== Ai Message ==================================

The average weight of a Border Collie is 37 lbs and the average weight of a Scottish Terrier is 20 lbs. Combined, their average weights would total 57 lbs. 

Let's now double this combined average weight.
Tool Calls:
  calculate (call_67f90d2bb18d4be1935345)
 Call ID: call_67f90d2bb18d4be1935345
  Args:
    what: 57 * 2
================================= Tool Message =================================
Name: calculate

114
================================== Ai Message ==================================

The doubled combined average weight of your Border Collie and Scottish Terrier is 114 lbs.


### 2.1.3 updates 模式

updates 模式下，流式输出的调用方式为：

In [15]:
for chunk in agent.stream(
 {
  "messages": [{"role": "user",
   "content": "I have 2 dogs, a border collie and a scottish terrier. What is their combined weight? Could you double it?"
  }] 
 },
 config={"configurable": {"thread_id": "user_1"}},
 stream_mode="updates" 
):
  for step, data in chunk.items():
    print(f"step: {step}")
    print(f"content: {data['messages'][-1].content_blocks}")

step: model
content: [{'type': 'tool_call', 'id': 'call_4ac37f11b4c14990921557', 'name': 'average_dog_weight', 'args': {'name': 'border collie'}}, {'type': 'tool_call', 'id': 'call_b293468aadd54c1f881573', 'name': 'average_dog_weight', 'args': {'name': 'scottish terrier'}}]
step: tools
content: [{'type': 'text', 'text': "A Border Collie's average weight is 37 lbs"}]
step: tools
content: [{'type': 'text', 'text': 'Scottish Terriers average 20 lbs'}]
step: model
content: [{'type': 'text', 'text': "The average weight of a Border Collie is 37 lbs, and the average weight of a Scottish Terrier is 20 lbs. Combined, their average weights total 57 lbs.\n\nNow, let's double this combined average weight."}, {'type': 'tool_call', 'id': 'call_6371a661f1364c40adee66', 'name': 'calculate', 'args': {'what': '57 * 2'}}]
step: tools
content: [{'type': 'text', 'text': '114'}]
step: model
content: [{'type': 'text', 'text': 'The doubled combined average weight of your Border Collie and Scottish Terrier is 

### 2.1.4 使用 ChatInterface 实现流式输出页面构建

在流式输出下，假如要构建一个 gradio 页面来动态进行展示，最好的选择就是使用 update 模式，因为其会将 model 和 tool 中每一步的决策都进行打印，总结来说就三类决策：
- 模型调用工具（step 为 model ，type 为 tool_call）
- 模型正常回复内容（step 为 model ， type 为 text）
- 工具执行结果（step 为 tools，type 为 text）

因此，根据这一特性，我们就可以制作不同的标签来进行个性化的显示。在 gradio 中只要在原有的传入信息中加上 metadata = {”title”:”...”}  即可实现标签添加，比如：

In [16]:
def generate_response(history):
    history.append(
        ChatMessage(role="assistant",
                    content="The weather API says it is 20 degrees Celcius in New York.",
                    metadata={"title": "🛠️ Used tool Weather API"}))
    return history

基于此，我们可以先定制一个 agent_response_stream_updates 函数（但不用直接更新 history，而是用一个新的历史来进行流式输出，不然 history 里会有上一次的聊天记录）：

In [ ]:
import gradio as gr
def agent_response_stream_updates(content, history):
    # 🚨 不使用 Gradio 传入的 history
    round_history = []

    inputs = {
        "messages": [{"role": "user", "content": content}]
    }

    config = {
        "configurable": {"thread_id": "user_1"}  # Agent 记忆仍然保留
    }

    for chunk in agent.stream(
        inputs,
        config=config,
        stream_mode="updates"
    ):
        for step, data in chunk.items():
            blocks = data["messages"][-1].content_blocks

            for block in blocks:
                block_type = block.get("type")

                # ===== 1. model -> tool_call =====
                if step == "model" and block_type == "tool_call":
                    round_history.append(
                        {
                            "role": "assistant",
                            "content": f"{block['name']}({block.get('args')})",
                            "metadata": {"title": "🧠 Tool call"}
                        }
                    )
                    yield round_history

                # ===== 2. tools -> text =====
                elif step == "tools" and block_type == "text":
                    round_history.append(
                        {
                            "role": "assistant",
                            "content": block["text"],
                            "metadata": {"title": "🛠️ Tool result"}
                        }
                    )
                    yield round_history

                # ===== 3. model -> text =====
                elif step == "model" and block_type == "text":
                    round_history.append(
                        {
                            "role": "assistant",
                            "content": block["text"]
                        }
                    )
                    yield round_history


然后把这个函数载入到 ChatInterface 中并启动：

In [18]:
import gradio as gr
demo = gr.ChatInterface(fn=agent_response_stream_updates)
demo.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


当我们正常提问并调用的话就会显示三种不同的对话模式了（具体请前往 gradio 运行代码中查看）。

## 2.2 结构化输出

在新版的 create_agent 中，我们还可以添加结构化输出的组件，使得最终返回的结果不再是字符串，而是一个 JSON 格式的内容。

和之前 LangChain 的格式化输出一样，我们需要先通过 Pydantic 定义一个基本的格式：

In [19]:
from pydantic import BaseModel, Field

class DogWeightSummary(BaseModel):
    """Structured summary of dog weight analysis."""
    dogs: list[str] = Field(description="List of dog breeds mentioned")
    combined_weight: float = Field(description="Total combined weight of the dogs in pounds")
    doubled_weight: float = Field(description="Double of the total combined weight")

这里我们就定义了要结构化返回的内容是列表格式的 dogs，浮点数格式的 combined_weight 和 doubled_weight ，并且都对应的添加了解释。

这个时候我们把这部分的内容载入到 create_agent 中：

In [20]:
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=system_prompt,
    checkpointer=memory,
    response_format=DogWeightSummary)

然后我们同样需要设置 thread_id 并将问题进行传入：

In [21]:
result1 = agent.invoke({"messages": [{"role": "user", "content": "I have 2 dogs, a border collie and a scottish terrier. What is their combined weight? Could you double it?"}]}, config={"configurable": {"thread_id": "user_1"}})
print(result1)

ValueError: request_id: b5596c73-511f-45bb-aadd-ab20d807efa0 
 status_code: 400 
 code: InvalidParameter 
 message: <400> InternalError.Algo.InvalidParameter: tool_choice is one of the strings that should be ["none", "auto"]

但是很可惜的是 ChatTongyi 并不支持 LangChain 的 Agent 结构化输出。假如想要体验的话，需要使用 OpenAI 或者 Anthropic 系列的模型：

In [ ]:
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-5-nano")

from langchain.agents import create_agent
agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=system_prompt,
    checkpointer=memory,
    response_format=DogWeightSummary)

result1 = agent.invoke({"messages": [{"role": "user", "content": "I have 2 dogs, a border collie and a scottish terrier. What is their combined weight? Could you double it?"}]}, config={"configurable": {"thread_id": "user_1"}})
print(result1)